In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
#from sklearn.cross_validation import train_test_split
# Building a english to french translator

In [0]:
lines= pd.read_table('/content/drive/My Drive/NLPProject/part10', names=['questions', 'keywords'])

In [0]:
#lines = lines[0:50000]
print(type(lines.keywords))
print(lines.keywords)
lines.keywords = lines.keywords.apply(lambda keywrds: " ".join([ele for ele in eval(keywrds)]))

<class 'pandas.core.series.Series'>
0            ['Replace', 'battery', 'key', 'jetta', 'gls']
1             ['Replace', 'battery', 'VW', 'jetta', 'key']
2         ['Replace', 'battery', 'VW', 'jetta', 'key', ...
3                              ['Replace', 'key', 'jetta']
4         ['Replace', 'keyless', 'remote', 'battery', '...
                               ...                        
12283              ['does', 'mountain', 'effect', 'earth']
12284     ['does', 'mountains', 'effect', 'earths', 'su...
12285     ['does', 'mountains', 'effect', 'earth', 'sur...
12286                      ['does', 'mountains', 'effect']
12287       ['does', 'rain', 'effect', 'earth', 'surface']
Name: keywords, Length: 12288, dtype: object


In [0]:
lines.shape

(12288, 2)

In [0]:
lines.head()

,questions,keywords
0,Replace battery in key of 2003 jetta gls?,Replace battery key jetta gls
1,Replace battery on 2006 VW jetta key?,Replace battery VW jetta key
2,Replace battery on VW jetta 2001 key remote?,Replace battery VW jetta key remote
3,Replace key 1999 jetta?,Replace key jetta
4,Replace keyless remote battery on volkswagen j...,Replace keyless remote battery volkswagen jetta


In [0]:
lines.questions=lines.questions.apply(lambda x: x.lower())
lines.keywords=lines.keywords.apply(lambda x: x.lower())

In [0]:
#removing the apostraphys and replacing comma(,) with COMMA
lines.questions=lines.questions.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))
lines.keywords=lines.keywords.apply(lambda x: re.sub("'", '', x)).apply(lambda x: re.sub(",", ' COMMA', x))

In [0]:
#removing the punctuation
punctu = set(string.punctuation)
lines.questions=lines.questions.apply(lambda x: ''.join(ch for ch in x if ch not in punctu))
lines.keywords=lines.keywords.apply(lambda x: ''.join(ch for ch in x if ch not in punctu))

In [0]:
#removing the digits
dig = str.maketrans('', '', digits)
lines.questions=lines.questions.apply(lambda x: x.translate(dig))
lines.keywords=lines.keywords.apply(lambda x: x.translate(dig))

In [0]:
lines.head()

,questions,keywords
0,replace battery in key of jetta gls,replace battery key jetta gls
1,replace battery on vw jetta key,replace battery vw jetta key
2,replace battery on vw jetta key remote,replace battery vw jetta key remote
3,replace key jetta,replace key jetta
4,replace keyless remote battery on volkswagen j...,replace keyless remote battery volkswagen jetta


In [0]:
lines.questions = lines.questions.apply(lambda x : 'START_ '+ x + ' _END')

In [0]:
all_ques_words = set()
for ques in lines.questions:
    for word in ques.split():
        if word not in all_ques_words:
            all_ques_words.add(word)
    
all_key_words = set()
for k in lines.keywords:
    for word in k.split():
        if word not in all_key_words:
            all_key_words.add(word)

In [0]:
len(all_ques_words), len(all_key_words)

(7739, 7220)

In [0]:
lenght_list=[]
for l in lines.questions:
    lenght_list.append(len(l.split(' ')))
maxqueslength = np.max(lenght_list)
maxqueslength

45

In [0]:
lenght_list=[]
for l in lines.keywords:
    lenght_list.append(len(l.split(' ')))
maxkeylength = np.max(lenght_list)
maxkeylength

22

In [0]:
input_words = sorted(list(all_key_words))
target_words = sorted(list(all_ques_words))
num_encoder_tokens = len(all_key_words)
num_decoder_tokens = len(all_ques_words)


In [0]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

In [0]:
len(lines.questions)*maxqueslength*num_decoder_tokens

4279357440

In [0]:
encoder_input_data = np.zeros(
    (len(lines.keywords), maxkeylength),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(lines.questions), maxqueslength),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(lines.questions), maxqueslength, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(lines.keywords, lines.questions)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

### Build keras encoder-decoder model

In [0]:
embedding_size = 50

In [0]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
from keras.models import load_model

Using TensorFlow backend.


#### Encoder model

In [0]:
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(units=50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#### Decoder model

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(units=50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

import os.path
from os import path
model = ""
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])


In [0]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 50)     361000      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     386950      input_2[0][0]                    
____________________________________________________________________________________________

In [0]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=300,
          validation_split=0.05 )

Train on 11673 samples, validate on 615 samples
Epoch 1/300
11673/11673 [==============================] - 22s 2ms/step - loss: 1.4768 - acc: 0.0285 - val_loss: 1.3850 - val_acc: 0.0297
Epoch 2/300
11673/11673 [==============================] - 22s 2ms/step - loss: 1.2961 - acc: 0.0298 - val_loss: 1.4051 - val_acc: 0.0297
Epoch 3/300
11673/11673 [==============================] - 22s 2ms/step - loss: 1.2836 - acc: 0.0313 - val_loss: 1.4158 - val_acc: 0.0336
Epoch 4/300
11673/11673 [==============================] - 22s 2ms/step - loss: 1.2748 - acc: 0.0330 - val_loss: 1.4209 - val_acc: 0.0339
Epoch 5/300
11673/11673 [==============================] - 22s 2ms/step - loss: 1.2666 - acc: 0.0337 - val_loss: 1.4279 - val_acc: 0.0308
Epoch 6/300
11673/11673 [==============================] - 22s 2ms/step - loss: 1.2591 - acc: 0.0346 - val_loss: 1.4330 - val_acc: 0.0343
Epoch 7/300
11673/11673 [==============================] - 22s 2ms/step - loss: 1.2518 - acc: 0.0353 - val_loss: 1.4363 - va

In [0]:
#model.save('/content/drive/My Drive/NLPProject/mytestmodel')

In [0]:
print(encoder_inputs)

Tensor("input_1:0", shape=(?, ?), dtype=float32)


In [0]:
print(encoder_states)

[<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 50) dtype=float32>]


In [0]:

encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 50)          361000    
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 381,200
Trainable params: 381,200
Non-trainable params: 0
_________________________________________________________________


In [0]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs = Input(shape=(None,))
final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [0]:
#print(len(encoder_input_data))
#for seq_index in [1407,2012,4003,4006, 4005, 4006, 4009, 4009, 4010, 4011, 4013, 4016, 4015, 4153]:
#all_key_words=set()
#all_ques_words=set()
with open('/content/drive/My Drive/NLPProject/TEST.txt','r') as testmodel:
  line = testmodel.readline()
  count = 1
  totalkeywords = []
  while line:
    words = [word.strip().lower() for word in line.split(' ')]
    print(words)
    answer  = words[-1]
    keywords = " ".join(words[:-1])
    print(answer," keyword:",keywords)
    totalkeywords.append([keywords,answer])
    count +=1
    if count == 100:
      break
    line = testmodel.readline()
# for k in totalkeywords:
#     for word in k[0].split():
#         if word not in all_test_key_words:
#             all_test_key_words.add(word)


['ticking', 'sound', 'alternator', 'starter', 'battery', 'starter']
starter  keyword: ticking sound alternator starter battery
['measurement', 'is', 'speed', 'wind', 'measured', 'knots']
knots  keyword: measurement is speed wind measured
['jim', 'morrison', 'died', 'yes']
yes  keyword: jim morrison died
['mexico', 'bigger', 'austraila', 'yes']
yes  keyword: mexico bigger austraila
['teachings', 'muslim', 'influenced', 'arts', 'artisticstyles']
artisticstyles  keyword: teachings muslim influenced arts
['bacterial', 'infections', 'cause', 'cancer', 'no']
no  keyword: bacterial infections cause cancer
['man', 'loves', 'woman', 'depends']
depends  keyword: man loves woman
['solar', 'energy', 'affect', 'weather', 'climate', 'heavyrainfall']
heavyrainfall  keyword: solar energy affect weather climate
['multiplex', 'control', 'unit', 'replace', 'honda', 'accord', 'reporgramming']
reporgramming  keyword: multiplex control unit replace honda accord
['smoking', 'affect', 'lung', 'capacity', 'bui

In [0]:
print(encoder_input_data)
print(encoder_input_data[2001: 2001 + 1])
print(encoder_input_data[2001])

[[5432.  553. 3514. ...    0.    0.    0.]
 [5432.  553. 6912. ...    0.    0.    0.]
 [5432.  553. 6912. ...    0.    0.    0.]
 ...
 [1909. 4214. 2061. ...    0.    0.    0.]
 [1909. 4214. 2061. ...    0.    0.    0.]
 [1909. 5283. 2061. ...    0.    0.    0.]]
[[ 338. 7218. 5898.    0.    0.    0.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]
[ 338. 7218. 5898.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]


In [0]:
f = open('GeneratedQAOutput.txt','w')
encoder_test_input_data = np.zeros((1, maxkeylength),dtype='float32')
for wordseq in totalkeywords:
    input_seq = wordseq[0]
    for t, word in enumerate(input_seq.split()):
        encoder_test_input_data[0, t] = input_token_index[word]
    decoded_sentence = decode_sequence(encoder_test_input_data[0:1])
    print('-', file=f)
    print('Input sentence:', input_seq, file=f)
    print('Decoded sentence:', decoded_sentence, file=f)
    print('Anwser:',wordseq[1], file=f)
f.close()

-
Input sentence: ticking sound alternator starter battery
Decoded sentence:  what to human part of the the battery is the battery
Anwser: starter
-
Input sentence: measurement is speed wind measured
Decoded sentence:  what measurement is wind speed measured east and in parallelwhat
Anwser: knots
-
Input sentence: jim morrison died
Decoded sentence:  jim jordans baseball what the s _END
Anwser: yes
-
Input sentence: mexico bigger austraila
Decoded sentence:  how much can chemistry be considered in water _END
Anwser: yes
-
Input sentence: teachings muslim influenced arts
Decoded sentence:  cool quad procedures of flower in a and life in spain
Anwser: artisticstyles
-
Input sentence: bacterial infections cause cancer
Decoded sentence:  how chemistry is a creme brulee in _END
Anwser: no
-
Input sentence: man loves woman
Decoded sentence:  about a soldier wear for a _END
Anwser: depends
-
Input sentence: solar energy affect weather climate
Decoded sentence:  how much will a great dane a co

### BLEU Score 

In [45]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
f = open('BleuScores.txt','w')
bleu_score = 0
input_tag = 'Input sentence:'
output_tag = 'Decoded sentence:'
with open('/content/GeneratedQAOutput.txt','r+',encoding='utf-8') as outputFile:
  num = 0
  score = 0
  sf = SmoothingFunction()
  for line in outputFile:
    if '-' in line:
      num += 1
      for i in range(3):
        try:
          line = next(outputFile)
          if input_tag in line:
            input_sequence = (line.replace(input_tag,'')).strip()
          elif output_tag in line:
            decoded_sentence = (line.replace(output_tag,'')).strip()
        except StopIteration:
          break
      score = sentence_bleu(input_sequence, decoded_sentence,smoothing_function = sf.method7)
      print('-', file=f)
      print('Input sentence:', input_sequence, file=f)
      print('Decoded sentence:', decoded_sentence, file=f)
      print('Bleu score : ',score, file=f)
      bleu_score += score
print("Cummulative Bleu Score obtained is : ",bleu_score/num)
f.close()

Cummulative Bleu Score obtained is :  0.3687531437076924
